# LCEL (LangChain Expression Language)
* 사용자의 입력 -> 프롬프트 템플릿 -> LLM 모델 전달
* 각 단계를 차례로 엮어서 하나의 파이프라인으로 만들어주는 것

chain = prompt | model | output_parser


### 프롬프트 탬플릿
* 사용자의 입력 변수를 받아서 완전한 프롬프트 문자열을 만드는데 사용하는 템플릿

In [4]:
from langchain_core.prompts import PromptTemplate

template = "{lecture}에 대해 설명해주세요"

prompt_template = PromptTemplate(template=template, input_variables=["lecture"])

In [5]:
lecture = input("궁금한 과목을 입력하세요 :")

prompt = prompt_template.format(lecture=lecture)

print(prompt)

스프링에 대해 설명해주세요


In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature=0.1
)

In [8]:
chain = prompt_template | model

In [10]:
# Prompt Template의 변수에 작성할 때는 Dictionary 형태로 key-value쌍으로 
# 입력을 줄 수 있다.
input = {"lecture":"랭체인"}

In [11]:
chain.invoke(input)

AIMessage(content='랭체인은 블록체인 기술을 기반으로 한 분산형 랭킹 시스템입니다. 랭체인은 사용자들이 서로에 대한 평가를 투표하고 이를 블록체인에 기록하여 신뢰할 수 있는 랭킹을 제공합니다. 이를 통해 사용자들은 신뢰할 수 있는 정보를 얻을 수 있고, 랭킹 시스템을 조작하는 행위를 방지할 수 있습니다. 또한, 랭체인은 다양한 분야에서 활용될 수 있으며, 신뢰성 있는 정보를 제공하는데 도움을 줄 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 18, 'total_tokens': 218, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1bb4f4cf-90a1-4cbc-8565-3c24bb2f993f-0', usage_metadata={'input_tokens': 18, 'output_tokens': 200, 'total_tokens': 218, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### 출력 형식 지정 (Output Parser)
* LLM모델이 응답해주는 내용에 대한 형식을 지정해주는 것

In [15]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [16]:
chain = prompt_template | model | output_parser

In [17]:
chain.invoke(input)

'랭체인은 블록체인 기술을 기반으로 한 분산형 랭킹 시스템입니다. 랭체인은 사용자들이 서로의 활동을 평가하고 순위를 매기는 시스템으로, 다양한 온라인 플랫폼에서 사용될 수 있습니다. \n\n랭체인은 중앙 집중식 시스템과는 달리 분산된 네트워크에서 운영되며, 블록체인 기술을 통해 데이터의 무결성과 투명성을 보장합니다. 사용자들은 서로의 활동을 평가하고 랭킹을 매기는 과정에서 보상을 받을 수 있으며, 이를 통해 더욱 활발한 활동을 유도할 수 있습니다.\n\n랭체인은 다양한 분야에서 활용될 수 있으며, 온라인 게임, 소셜 미디어, 온라인 교육 등 다양한 플랫폼에서 사용자들 간의 상호작용을 촉진하고 보상을 제공하는데 활용될 수 있습니다. 또한, 랭체인은 사용자들이 신뢰할 수 있는 정보를 얻을 수 있도록 도와주고, 플랫폼의 품질을 향상시키는데 기여할 수 있습니다.'

In [18]:
### 영어 회화를 알려주는 선생님 챗봇
### 어떤 상황을 주면 영어회화 + 한글해석

template ="""
당신은 영어를 가르치는 10년차 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해주세요.
양식은 [FORMAT]을 참고하여 작성해주세요

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿 작성
prompt = PromptTemplate.from_template(template)

# 모델 객체 작성
model = ChatOpenAI(model_name="gpt-4o")

# 문자열 출력 파서 객체 생성
output_parser = StrOutputParser()


In [19]:
# 체인 구성

chain = prompt | model | output_parser

In [20]:
# Chain 실행
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  - Customer: Hello! Can I see the menu, please?
  - Waiter: Of course! Here you go. Do you have any questions about the menu?
  - Customer: Thank you. Yes, I have a question. What do you recommend?
  - Waiter: Our grilled salmon is very popular, and the pasta is also a favorite.
  - Customer: That sounds great! I’ll have the grilled salmon, please.
  - Waiter: Certainly. Would you like any appetizers or drinks with that?
  - Customer: Yes, I’ll have a garden salad and a glass of iced tea.
  - Waiter: Perfect. I’ll get that started for you. It should be ready soon.
  - Customer: Thank you!

- 한글 해석:
  - 손님: 안녕하세요! 메뉴를 볼 수 있을까요?
  - 웨이터: 물론입니다! 여기 있습니다. 메뉴에 대해 궁금한 점 있으신가요?
  - 손님: 감사합니다. 네, 질문이 있어요. 추천해 주실 만한 게 있나요?
  - 웨이터: 저희의 구운 연어가 매우 인기 있고, 파스타도 많이 좋아하세요.
  - 손님: 좋네요! 구운 연어로 할게요.
  - 웨이터: 알겠습니다. 그에 곁들일 전채나 음료는 필요 없으세요?
  - 손님: 네, 가든 샐러드와 아이스티 한 잔 주세요.
  - 웨이터: 알겠습니다. 준비해 드리겠습니다. 곧 나올 겁니다.
  - 손님: 감사합니다!


여러가지 재료를 변수를 입력 받고, 재료로 요리할 수 있는 <br>
요리의 요리명, 재료, 조리법을 포맷으로 받는 템플릿을 작성하고 <br>
문자열 출력 파서로 응답을 받을 수 있게 체인을 구성하세요 <br>

In [21]:

template = """
당신은 현재 여러가지 재료를 가지고 요리를 해야 합니다. 주어진 재료에 맞는 요리를 작성해주세요
양식은 [FORMAT]을 참고하여 작성해주세요.

#상황:
{question}

#FORMAT:
- 요리명:
- 재료:
- 조리법:

"""

prompt = PromptTemplate.from_template(template)

model = ChatOpenAI(tiktoken_model_name="gpt-4o")

output_parser = StrOutputParser()

In [22]:
chain = prompt | model| output_parser

In [24]:
print(chain.invoke({"question":"요리를 하는 법을 알려주세요"}))

#상황:
당신은 감자, 당근, 양파, 소고기를 가지고 있습니다. 이 재료들로 요리를 해보려고 합니다.

#FORMAT:
- 요리명: 감자 당근 소고기 볶음
- 재료: 감자, 당근, 양파, 소고기, 간장, 설탕, 식용유
- 조리법: 
1. 감자와 당근은 깍뚝 썰고, 양파는 채썰어 준비합니다.
2. 소고기는 얇게 썬 후 간장과 설탕을 넣고 조물조물 버무려 준비합니다.
3. 팬에 식용유를 두르고 양파를 볶다가 소고기를 넣고 익힙니다.
4. 소고기가 익으면 감자와 당근을 넣고 볶아줍니다.
5. 감자와 당근이 익으면 소스를 넣고 한번 더 볶아주면 완성입니다.마늘을 넣고 볶아줍니다.
6. 마늘이 향이 올라오면 불을 끄고 고명을 뿌려 완성합니다.
